In [1]:
from bs4 import BeautifulSoup
import unicodedata
import requests
import re
from sqlalchemy import create_engine, Column, String, Integer, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [2]:
#init connection to sqlite 
engine = create_engine("sqlite:///events.db")
#create session to cache commands for sqlite engine instance
Session = sessionmaker(bind = engine)
session = Session()

In [3]:
#provide table definition
Base = declarative_base()

class Event(Base):
    __tablename__ = 'event'
    id = Column('id',Integer, primary_key = True)
    district = Column(Integer)
    title = Column(String(100))
    date = Column(String(50))
    details = Column(String(1000))
    time = Column(String(50))

    def __init__(self, title, date, details,time,district):
        self.title = title
        self.date = date
        self.details = details
        self.time = time
        self.district = district
        
    #for print    
    def __repr__(self):
        return f'{self.title} - {self.date}: {self.time}\n {self.details}'

In [4]:
#call to metadata to generate schema
Base.metadata.create_all(engine)

In [5]:
page = requests.get('https://www1.nyc.gov/site/manhattancb1/meetings/committee-agendas.page')

In [6]:
soup = BeautifulSoup(page.text,'html.parser')

In [7]:
paragraphs = soup.find(class_='about-description').find_all('p')

In [8]:
events = []
event_num = None
event = []

for par in paragraphs:
    #ends event and starts a new event
    if event_num is not None and re.search(r"^(\d*[/]\d*)",par.text):
        events.append(event)
        event = []
        event_num += 1

    #finds intial event and starts the process
    if event_num is None and re.search(r"^(\d*[/]\d*)",par.text):
        event = []
        event_num = 0

    event.append(par)


In [9]:
events_dict = {}

for i, event in enumerate(events):
    event_string = ''.join([tag.text for tag in event])
    #print(event_string, '\n')
    date = re.search(r"^(\d*[/]\d*)",event_string).group(0)
    title = event[0].find('b').text
    details = ''.join([tag.text for tag in event[1:]])
    details = details.strip().replace('\xa0', '')
    time = re.search(r"((1[0-2]|0?\d):(\d\d) ([AaPp][Mm]))",event_string)  
        
    events_dict[i] = {
        'date': date,
        'time':time.group(0) if time else '',
        'title': title,
        'details':details
    }

In [18]:
district = 105
#remove previous entries
session.query(Event).filter(Event.district == district).delete()
session.commit()

#add items to database
for event in events_dict.values():
    row = Event(title=event['title'], date=event['date'],
                details=event['details'], time=event['time'], district= district)
    session.add(row)
session.commit()

In [19]:
#print all users
for event in session.query(Event).filter(Event.district == 101):
    print(event)

Transportation & Street Activity Permits Committee – 6:00 PM - 5/5: 6:00 PM
Battery Park City Committee – 6:00 PM - 5/6: 6:00 PM
Land Use, Zoning & Economic Development Committee – 6:00PM - 5/11: 
Youth & Education Committee – 5:30 PM         (Held Jointly with Manhattan CB 2’s Youth Committee) - 5/12: 5:30 PM
Licensing & Permits Committee – 6:00 PM - 5/13: 6:00 PM
Landmarks & Preservation Committee – 6:00 PM - 5/14: 6:00 PM
Environmental Protection Committee – 6:00 PM - 5/18: 6:00 PM
Waterfront, Parks & Cultural Committee – 6:00 PM - 5/19: 6:00 PM
Executive Committee – 6:00 PM - 5/20: 6:00 PM
Quality of Life & Service Delivery Committee – 6:00 PM - 5/21: 6:00 PM
Office Closed – Memorial Day - 5/25: 
